# Title: csck700_ifc_parser

### The module parses an IFC model to extract the IFcElements, their properties and relationships and loads them to nodes and directed edge into Neo4J.

# 0. Table of content:

# 1. Settings:

In [6]:
IFC_PATH = r"..\data\raw\Building-Structural.ifc"

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "TitineTiteFiro@1952"

In [60]:
REL_MAP ={
    "IfcRelDefinesByProperties": ("RelatingPropertyDefinition", "RelatedObjects", None),
    "IfcRelDefinesByType": ("RelatingType", "RelatedObjects", "DEFINED_BY_TYPE"),
    "IfcRelAssociatesMaterial": ("RelatingMaterial", "RelatedObjects", "ASSOCIATED_MATERIAL"),
    "IfcRelContainedInSpatialStructure": ("RelatingStructure", "RelatedElements", "CONTAINED_IN"),
    "IfcRelAggregates": ("RelatingObject", "RelatedObjects", "AGGREGATES"),
    "IfcRelAssociatesClassification": ("RelatingClassification", "RelatedObjects", "ASSOCIATED_CLASSIFICATION")
} 

# 2. Imports:

In [9]:
import ifcopenshell
import ifcopenshell.util.element as util
from neo4j import GraphDatabase
from collections import Counter

# 3. Helper functions:

In [11]:
def elements_inspection(model, ifc_type="IfcProduct"):
    """
    Counts and lists IfcElements (excluding proxies in this case) grouped by class.
    Returns:
        - elements: list.
        - classes_counter: Counter.
    """
    
    products = model.by_type(ifc_type)
    elements = [p for p in products if (p.is_a("IfcElement")) and not (p.is_a("IfcBuildingElementProxy"))]
    elements_classes = [el.is_a() for el in elements]
    classes_counter = Counter(elements_classes)

    print(f"Total number of elements: {len(elements)}")
    for cl, count in classes_counter.items():
        print(f"{cl}: {count}.")

    return elements, classes_counter

In [12]:
def related_nodes_maker(elements):
    """
    Build a dictionary of IfcElements nodes with basic properties.
    Returns:
        - related_nodes: dictionary with keys:
            id, labels, name, predefined_type
    """

    related_nodes = dict()
    
    for element in elements:
        related_nodes[element.GlobalId] = {
            "id": element.GlobalId,
            "labels": ["Element", element.is_a()],
            "name": element.Name,
            "predefined_type": element.ObjectType
    }
    
    print(f"Total number of related nodes: {len(related_nodes)}")

    return related_nodes

In [13]:
# def rel_identify(model, elements):
#     """
#     Identifies the IfcRelationship classes applied to the given IfcElements.
#     Returns:
#         - Set: names of the IfcRelationship classes found.
#     """
    
#     rel_classes = set()
    
#     for element in elements:
#         element_inverse = model.get_inverse(element)
#         for rel in element_inverse:
#             if rel.is_a().startswith("IfcRel"):
#                 rel_classes.add(rel.is_a())
                
#     print("Identified relationship classes:")  
#     for r in rel_classes:
#         print(f"-{r}.")
        
#     return rel_classes

In [44]:
def identify_rel(model):
    """
    Identify all distinct IfcRelationship classes present in the model.
    """

    rel_classes = set()

    rels = model.by_type("IfcRelationship")

    for rel in rels:
        rel_classes.add(rel.is_a())

    print("Identified relationship classes:")  
    for r in rel_classes:
        print(f"- {r}.")
        
    return rel_classes

In [50]:
def obj_id(o):
    """
    Returns the GUID of the given object, if present, else synthetic ClassName:step_id.

    Non-IfcRoot entities (e.g. IfcMaterial, IfcClassificationReference)
    do not have a GlobalId, so the fallback ensures uniqueness.
    """
    return getattr(o, "GlobalId", f"{o.is_a()}:{o.id()}")

In [52]:
def as_list(v):
    """
    Ensures the value is returned as a list.
    """
    if v is None:
        return []
    return v if isinstance(v, (list, tuple)) else [v]

In [62]:
def edges_maker(model, rel_map=REL_MAP, pset_qto=True):
    """
    For a given model, creates a dictionary of edges (IfcRelationships exploded into from -> to pairs.
    Returns:
        - edges: a dictionary of edges with keys:
            - id, labels, rel_class, rel_id, from, to.
    """

    edges = dict()
    pairs = set()

    for rel_class, (relating_attr, related_attr, label) in rel_map.items():
        for rel in model.by_type(rel_class):
            relating = getattr(rel, relating_attr, None)
            related = getattr(rel, related_attr, None)
            if relating is None:
                continue
            
            # distinction property set vs quantity set:
            if rel_class == "IfcRelDefinesByProperties":
                if pset_qto and relating.is_a("IfcElementQuantity"):
                    edge_label = "DEFINED_BY_QUANTITIES"
                else:
                    edge_label = "DEFINED_BY_PROPERTIES"
            else:
                edge_label = label

            frm = obj_id(relating)
            rel_gid = obj_id(rel)

            # one-to-many relationships:
            for r in as_list(related):
                if r is None:
                    continue
                to  = obj_id(r)
                
                # sanity check:
                pairs.add((rel_gid, frm, to))
                
                edge_id = f"{rel_gid}:{frm}->{to}"
                edges[edge_id]={
                    "id": edge_id,
                    "labels": edge_label,
                    "rel_class": rel_class,
                    "rel_id": rel_gid,
                    "from": frm,
                    "to": to
                }
    
    print(f"Expected number of edges: {len(pairs)}.")
    print(f"Number of edges: {len(edges)}.")
    
    return edges

# 4. Data load:

In [18]:
model = ifcopenshell.open(IFC_PATH)

In [19]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# 5. IFC Parsing:

## 5.1. Related nodes:

In [22]:
elements, classes = elements_inspection(model)

Total number of elements: 15
IfcBeam: 6.
IfcChimney: 1.
IfcFooting: 1.
IfcRoof: 1.
IfcWall: 4.
IfcDiscreteAccessory: 2.


In [23]:
related_nodes = related_nodes_maker(elements)

Total number of related nodes: 15


## 5.2. Relationship edges:

### IFC inverse relationships are not included in the Neo4j graph. Neo4j can traverse edges in both directions, so storing inverse relationships would only duplicate data and waste memory.  

In [56]:
identify_rel(model)

Identified relationship classes:
- IfcRelDefinesByProperties.
- IfcRelDefinesByType.
- IfcRelAssociatesClassification.
- IfcRelContainedInSpatialStructure.
- IfcRelAggregates.
- IfcRelAssociatesMaterial.


{'IfcRelAggregates',
 'IfcRelAssociatesClassification',
 'IfcRelAssociatesMaterial',
 'IfcRelContainedInSpatialStructure',
 'IfcRelDefinesByProperties',
 'IfcRelDefinesByType'}

In [64]:
edges = edges_maker(model)

Expected number of edges: 79.
Number of edges: 79.


In [66]:
edges

{'141p4IPfrBnRJ5dWbakDZK:18DB$FpSH4QhNntrrSRVGz->0c$N1CTon2BB2Sp89385G8': {'id': '141p4IPfrBnRJ5dWbakDZK:18DB$FpSH4QhNntrrSRVGz->0c$N1CTon2BB2Sp89385G8',
  'labels': 'DEFINED_BY_PROPERTIES',
  'rel_class': 'IfcRelDefinesByProperties',
  'rel_id': '141p4IPfrBnRJ5dWbakDZK',
  'from': '18DB$FpSH4QhNntrrSRVGz',
  'to': '0c$N1CTon2BB2Sp89385G8'},
 '37JtmMF1HE8uDDbwzFG43E:0hVJYXG1r7ywQAq8Vpug40->0DyViLJJ175RvWQi1rE7a6': {'id': '37JtmMF1HE8uDDbwzFG43E:0hVJYXG1r7ywQAq8Vpug40->0DyViLJJ175RvWQi1rE7a6',
  'labels': 'DEFINED_BY_PROPERTIES',
  'rel_class': 'IfcRelDefinesByProperties',
  'rel_id': '37JtmMF1HE8uDDbwzFG43E',
  'from': '0hVJYXG1r7ywQAq8Vpug40',
  'to': '0DyViLJJ175RvWQi1rE7a6'},
 '1SWTv4zQ95LhMduNc2qS5N:0VVWsZ$_bFHgmAwNtcydte->0DyViLJJ175RvWQi1rE7a6': {'id': '1SWTv4zQ95LhMduNc2qS5N:0VVWsZ$_bFHgmAwNtcydte->0DyViLJJ175RvWQi1rE7a6',
  'labels': 'DEFINED_BY_QUANTITIES',
  'rel_class': 'IfcRelDefinesByProperties',
  'rel_id': '1SWTv4zQ95LhMduNc2qS5N',
  'from': '0VVWsZ$_bFHgmAwNtcydte',
  '

## 5.3. Relating nodes:

In [30]:
# def get_relating_object(model, oid):

#     if isinstance(oid, str) and (len(oid) == 22) and (":" not in oid):
#         return model.by_guid(oid)

#     elif ":" in oid:
#         try:
#             step_id = int(oid.split(":")[-1])
#             return model.by_id(step_id)
#         except Exception:
#             return None

#     return None


In [84]:
def get_object_from_id(model, oid):

    if isinstance(oid, str) and (len(oid) == 22) and (":" not in oid):
        return model.by_guid(oid)

    elif ":" in oid:
        try:
            step_id = int(oid.split(":")[-1])
            return model.by_id(step_id)
        except Exception:
            return None

    return None

In [86]:
def get_objects_from_edges(model, edges, typ):

    objects_from_edges = []
    
    for edge in edges.values():
        obj = get_object_from_id(model, edge[typ])
        if obj:
            objects_from_edges.append(obj) 

    return objects_from_edges

In [88]:
relating_objects = get_objects_from_edges(model, edges, "from")

In [90]:
related_objects = get_objects_from_edges(model, edges, "to")

In [92]:
relating_objects

[#32=IfcPropertySet('18DB$FpSH4QhNntrrSRVGz',#1,'Pset_BuildingCommon',$,(#31)),
 #76=IfcPropertySet('0hVJYXG1r7ywQAq8Vpug40',#1,'Pset_WallCommon',$,(#72,#74,#75)),
 #86=IfcElementQuantity('0VVWsZ$_bFHgmAwNtcydte',#1,'Qto_WallBaseQuantities',$,$,(#82,#83,#84,#85)),
 #105=IfcPropertySet('0ek$CCUjnFefFg04xFxxG_',#1,'Pset_WallCommon',$,(#102,#103,#104)),
 #111=IfcElementQuantity('22KfX2yaj4UBecOFxQr$3y',#1,'Qto_WallBaseQuantities',$,$,(#107,#108,#109,#110)),
 #128=IfcPropertySet('2DaAsqJGPDexb$sFANyw0e',#1,'Pset_WallCommon',$,(#126,#127)),
 #134=IfcElementQuantity('16mzz5kV15mAyMAccFRkj4',#1,'Qto_WallBaseQuantities',$,$,(#130,#131,#132,#133)),
 #176=IfcPropertySet('3UegLqoYrB5QLJ1Fl0C0gk',#1,'Pset_WallCommon',$,(#173,#174,#175)),
 #182=IfcElementQuantity('3BkYHDW2b4LxsPRJLXlKa9',#1,'Qto_WallBaseQuantities',$,$,(#178,#179,#180,#181)),
 #199=IfcPropertySet('3IC75ptmHAcfynDO0H7l6Q',#1,'Pset_RoofCommon',$,(#197,#198)),
 #213=IfcPropertySet('2ups0JG6n1VvC4pdB0zluW',#1,'Pset_BeamCommon',$,(#210,

In [94]:
related_objects

[#30=IfcBuilding('0c$N1CTon2BB2Sp89385G8',#1,'Single-family house','The main building structure, providing shelter and space.',$,#38,$,'house - building',.ELEMENT.,$,$,$),
 #71=IfcWall('0DyViLJJ175RvWQi1rE7a6',#1,'house - outer wall - house back','A solid outer wall, forming the back of the house.','solidwall',#88,#98,'454425.1027891.979946.932083.920030',$),
 #71=IfcWall('0DyViLJJ175RvWQi1rE7a6',#1,'house - outer wall - house back','A solid outer wall, forming the back of the house.','solidwall',#88,#98,'454425.1027891.979946.932083.920030',$),
 #101=IfcWall('3SGBcf7Lv0r80vKtUCgOpf',#1,'house - outer wall - house front','A solid outer wall, forming the front of the house.','solidwall',#113,#122,'454425.1027891.979946.932083.920035',$),
 #101=IfcWall('3SGBcf7Lv0r80vKtUCgOpf',#1,'house - outer wall - house front','A solid outer wall, forming the front of the house.','solidwall',#113,#122,'454425.1027891.979946.932083.920035',$),
 #125=IfcWall('3oNJ9yHi5FJuFnK8yg68Yt',#1,'house - outer w

In [100]:
len(relating_objects)

79

In [102]:
len(related_objects)

79

In [104]:
def merge_unique(obj1, obj2):
    merged = dict()
    for o in obj1 + obj2:
        if o:
            merged[obj_id(o)] = o
    return list(merged.values())

In [172]:
all_objects = merge_unique(relating_objects, related_objects)
print(len(all_objects))

64


In [174]:
print(all_objects)

[#32=IfcPropertySet('18DB$FpSH4QhNntrrSRVGz',#1,'Pset_BuildingCommon',$,(#31)), #76=IfcPropertySet('0hVJYXG1r7ywQAq8Vpug40',#1,'Pset_WallCommon',$,(#72,#74,#75)), #86=IfcElementQuantity('0VVWsZ$_bFHgmAwNtcydte',#1,'Qto_WallBaseQuantities',$,$,(#82,#83,#84,#85)), #105=IfcPropertySet('0ek$CCUjnFefFg04xFxxG_',#1,'Pset_WallCommon',$,(#102,#103,#104)), #111=IfcElementQuantity('22KfX2yaj4UBecOFxQr$3y',#1,'Qto_WallBaseQuantities',$,$,(#107,#108,#109,#110)), #128=IfcPropertySet('2DaAsqJGPDexb$sFANyw0e',#1,'Pset_WallCommon',$,(#126,#127)), #134=IfcElementQuantity('16mzz5kV15mAyMAccFRkj4',#1,'Qto_WallBaseQuantities',$,$,(#130,#131,#132,#133)), #176=IfcPropertySet('3UegLqoYrB5QLJ1Fl0C0gk',#1,'Pset_WallCommon',$,(#173,#174,#175)), #182=IfcElementQuantity('3BkYHDW2b4LxsPRJLXlKa9',#1,'Qto_WallBaseQuantities',$,$,(#178,#179,#180,#181)), #199=IfcPropertySet('3IC75ptmHAcfynDO0H7l6Q',#1,'Pset_RoofCommon',$,(#197,#198)), #213=IfcPropertySet('2ups0JG6n1VvC4pdB0zluW',#1,'Pset_BeamCommon',$,(#210,#211,#212)

In [187]:
def objects_classes_identify(all_objects):

    clss = set()
    
    for o in all_objects:
        if o.is_a("IfcElement"):
            clss.add("IfcElement")
        elif "Type" in o.is_a():
            clss.add("IfcTypeObject")
        else:
            clss.add(o.is_a())

    return clss

In [189]:
objects_classes_identify(all_objects)

{'IfcBuilding',
 'IfcBuildingStorey',
 'IfcClassificationReference',
 'IfcElement',
 'IfcElementQuantity',
 'IfcMaterial',
 'IfcProject',
 'IfcPropertySet',
 'IfcSite',
 'IfcTypeObject'}

In [181]:
# def props_identify(all_objects):

#     props = {}

#     pset = "IfcPropertySet"
#     qset = "IfcElementQuantity"

#     for o in all_objects:
#         if o.is_a(pset):
#             props.setdefault(pset, set())
#             for p in o.HasProperties:
#                 props[pset].add(p.Name)
                    
#         elif o.is_a(qset):
#             props.setdefault(qset, set())
#             for q in o.Quantities:
#                 props[qset].add(q.Name)

#     # only IfcMaterial class, no specific material properties to extract beyond the material name.
                
#     return props

In [207]:
def props_identify(all_objects):

    props = {}

    pset = "IfcPropertySet"
    qset = "IfcElementQuantity"

    for o in all_objects:
        if o.is_a(pset):
            props.setdefault(pset, dict())
            for p in o.HasProperties:
                props[pset][p.Name] = {
                    "property_type": p.is_a()
                }
                
        elif o.is_a(qset):
            props.setdefault(qset, dict())
            for q in o.Quantities:
                props[qset][q.Name] = {
                    "property_type": q.is_a()
                }

    # only IfcMaterial class, no specific material properties to extract beyond the material name.
                
    return props

In [209]:
props = props_identify(all_objects)
qto_props = list(props["IfcElementQuantity"])
qto_props

['NetVolume', 'Width', 'Length', 'NetSideArea', 'CrossSectionArea']

In [211]:
props

{'IfcPropertySet': {'ConstructionMethod': {'property_type': 'IfcPropertySingleValue'},
  'Status': {'property_type': 'IfcPropertyEnumeratedValue'},
  'IsExternal': {'property_type': 'IfcPropertySingleValue'},
  'LoadBearing': {'property_type': 'IfcPropertySingleValue'}},
 'IfcElementQuantity': {'NetVolume': {'property_type': 'IfcQuantityVolume'},
  'Width': {'property_type': 'IfcQuantityLength'},
  'Length': {'property_type': 'IfcQuantityLength'},
  'NetSideArea': {'property_type': 'IfcQuantityArea'},
  'CrossSectionArea': {'property_type': 'IfcQuantityArea'}}}

In [213]:
def pset_props_collect(o):

    def unwrap(v):
        if v is None:
            return None
        else:
            return getattr(v, "wrappedValue", v)

    props = {}
    
    for p in o.HasProperties:
        if p.is_a("IfcPropertySingleValue"):
            nv = getattr(p, "NominalValue", none)
            props[p.Name]= {
                "kind": "SingleValue",
                "value": unwrap(nv)
        elif p.is_a("IfcPropertyEnumeratedValue"):
            ......

    return props

SyntaxError: invalid syntax (1367782974.py, line 13)

In [142]:
def node_set(o):

    nodes = dict()

    def g_n(o, name, default=None):
        return getattr(o, name, default)

    cls = o.is_a()

    if cls == "IfcProject":
        nodes[obj_id(o)] = {
            "id": obj_id(o),
            "labels": ["PROJECT", cls],
            "name": g_n(o, "Name"),
        }

    elif o.is_a("IfcElement"):
        nodes[obj_id(o)] = {
            "id": obj_id(o),
            "labels": ["Element", o.is_a()],
            "name": g_n(o, "Name"),
            "predefined_type": getattr(o, "ObjectType", None)
        }

    elif csl == "IfcPropertySet":
        nodes[obj_id(o)] = {
            "id": obj_id(o),
            "labels": ["PSET", cls],
            "name": g_n(o, "Name"),
            "properties": {}
        }
    elif cls == "IfcElementQuantity":
        nodes[obj_id(o)] = {
            "id": obj_id(o),
            "labels": ["QSET", cls],
            "name": g_n(o, "Name"),
            "properties": {}
        }
    elif "Type" in cls:
        nodes[obj_id(o)] = {
            "id": obj_id(o),
            "labels": ["TYPE", cls],
            "name": g_n(o, "Name")
        }
    elif cls == "IfcMaterial":
        nodes[obj_id(o)] = {
            "id": obj_id(o),
            "labels": ["MATERIAL", cls],
            "name": g_n(o, "Name")
        }
    elif cls in ("IfcSite", "IfcBuilding", "IfcBuildingStorey", "IfcSpace"):
        nodes[obj_id(o)] = {
            "id": obj_id(o),
            "labels": ["SPATIAL", cls],
            "name": g_n(o, "Name")
        }
    # RelatingClassification in IfcRelassociatesClassification can point to:
    # - IfcClassificationReference
    # - IfcClassification
    
    elif cls == "IfcClassificationReference":
        scheme = g_n(o, "ReferencedSource") # IfcClassification
        nodes[obj_id(o)]:{
            "id": obj_id(o),
            "labels": ["CLASSIFICATION_REF", cls],
            "name": g_n(o, "Name"),
            "code": g_n(o, "Identification"),
            "uri": g_n(o, "Location"),
            # link to IfcClassification by id:
            "scheme_id": obj_id(scheme) if scheme else None,
        }
    # elif cls == "IfcClassification":
    #     nodes[obj_id(o)]:{
    #         "id": obj_id(o),
    #         "labels": ["CLASSIFICATION", cls],
    #         "name": g_n(o, "Name"),
    #         "publisher": g_n(o, "Source"),
    #         "edition": g_n(o, "Edition"),
    #         "edition_date": str(g_n(o, "EditionDate")) if g_n(o, "EditionDate") else None
    #     }

    return nodes        

In [ ]:
def nodes_maker(all_objects):
    

In [ ]:
for qset in ifc_file.by_type("IfcElementQuantity"):
    print("Qto name:", qset.Name)
    for q in qset.Quantities:
        print("   Quantity name:", q.Name, "| type:", q.is_a())
        
        # value depends on the subtype
        if q.is_a("IfcQuantityLength"):
            print("      Value:", q.LengthValue)
        elif q.is_a("IfcQuantityArea"):
            print("      Value:", q.AreaValue)
        elif q.is_a("IfcQuantityVolume"):
            print("      Value:", q.VolumeValue)
        elif q.is_a("IfcQuantityCount"):
            print("      Value:", q.CountValue)
        elif q.is_a("IfcQuantityWeight"):
            print("      Value:", q.WeightValue)

# 6. Neo4J graph: